In [5]:
import numpy as np
import pickle
import os

In [11]:
path = '../data/cifar-10-batches-py'
train = !ls $path | grep data
test = !ls $path | grep test

In [12]:
def load_file(fn):
    with open(fn, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
    return data

In [13]:
def load_batch(batch_id):
    return load_file(os.path.join(path, batch_id))

In [14]:
def load_CIFAR10():
    dicts_train = [load_batch(train_file) for train_file in train]
    dicts_test = [load_batch(test_file) for test_file in test]
    X_train_l, Y_train_l = zip(*[(d[b'data'], d[b'labels']) for d in dicts_train])
    X_test_l, Y_test_l = zip(*[(d[b'data'], d[b'labels']) for d in dicts_test])
    def concat(data_l): return np.concatenate([data for data in data_l], axis=0)
    return map(concat, [X_train_l, Y_train_l, X_test_l, Y_test_l])

In [28]:
Xtr, Ytr, Xte, Yte = load_CIFAR10()
Xtr_rows, Xte_rows = [X.reshape(X.shape[0], 32 * 32 * 3) for X in [Xtr, Xte]]

In [62]:
class NearestNeighbor:
    def __init__(self):
        pass
    
    def train(self, X, Y):
        self.Xtr = X
        self.Ytr = Y

    def predict(self, X):
        num_test = X.shape[0]
        Ypred = np.zeros(X.shape[0], dtype=self.Ytr.dtype)
        for i in range(num_test):
#             distances = np.sum(np.abs(self.Xtr - X[i,:]), axis=1)
            distances = np.sqrt(np.sum(np.square(self.Xtr - X[i, :]), axis=1))
            min_index = np.argmin(distances)
            Ypred[i] = self.Ytr[min_index]
        return Ypred

In [63]:
import time
nn = NearestNeighbor()
nn.train(Xtr_rows, Ytr)
start = time.time()
Yte_predict = nn.predict(X_test[:100])
print(f"accuracy: {np.mean(Yte_predict[:100] == Yte[:100])}")
print(f"took {time.time()-start}s")

accuracy: 0.29
took 24.32280445098877s
